In [1]:
#Libraries
import numpy as np
import numba as nb
from joblib import Parallel, delayed
import random
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
# Integrator of GLV dynamics

#RK4
@nb.jit
def RK4_step(x_t, S, K, alpha, dt):

    k1 = rhs(x_t, S, K, alpha)
    k2 = rhs(x_t + dt/2 * k1, S, K, alpha)
    k3 = rhs(x_t + dt/2 * k2, S, K, alpha)
    k4 = rhs(x_t + dt * k3, S, K, alpha)
    
    return x_t + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

@nb.jit
def rhs(x_t, S, K, alpha):
    f = np.zeros(S)
    for i in range(S):
        tmp = 0
        for j in range(S):
            tmp = tmp + alpha[i, j]* J(x_t[j], K)
    
        f[i] = x_t[i]*(1-x_t[i]+tmp)
    return f

@nb.jit
def RK4(S, alpha, K, timesteps, dt):
    x = np.zeros((S, timesteps))
    x[:, 0] = np.random.rand(S)
    for n in range(1, timesteps):
        x[:, n] = RK4_step(x[:, n-1], S, K, alpha, dt)
    return x


def inter_matrix(N, mu, sigma, gamma):
#covariance matrix and mean for sampling          
    alpha = np.zeros([N, N]) #matrix of interactions

    for i in range(N):
        for j in range(i+1, N):
            z = np.random.normal(size = 2)
            alpha[i, j] = mu/N+sigma/np.sqrt(N)*z[0]
            alpha[j, i] = mu/N+sigma/np.sqrt(N)*(gamma*z[0]+np.sqrt(1-gamma**2)*z[1])

    return alpha


@nb.njit
def J(x, K):
    return x/(1+K*x)


#task
def simulation(par):
    
    [S, mu, sigma, gamma, K, i] = par
       

    alpha = inter_matrix(S, mu, sigma, gamma)
    run1 = RK4(S, alpha, K, timesteps, dt)
    run2 = RK4(S, alpha, K, timesteps, dt)
    
    stat1 = run1[:, N_cut:]
    stat2 = run2[:, N_cut:]

    fold1 = 'S{}/'.format(S)
    fold2 = 'mu{}_sigma{}_gamma{}_K{}/'.format(mu, sigma, gamma, K)
    filename =  'mu{}_sigma{}_gamma{}_K{}_iter{}'.format(mu, sigma, gamma, K, i)       

    if SAVE_SIM:
        np.savez(path_to_save_sim+fold0+fold1+fold2+filename, alpha = alpha, run1=run1, run2=run2)
    
    if SAVE_STAT:
        np.savez(path_to_save_stat+fold0+fold1+fold2+filename, stat1=stat1, stat2=stat2)

        
    new_row = [S, mu, sigma, gamma, K]

    return new_row


In [9]:
#Parameters lists
S_list = [200]
mu_list = [-3]
sigma_list = [2.5, 2.7, 3.1, 3.3, 2.9, 2.3, 3.5, 3.7, 2.1]
gamma_list = [-0.5]
K_list = [0.1]

In [10]:
# Simulation parameters
N_sam = 20 #number of sampling of interaction matrix
dt = 0.005
timesteps = 40_000
N_cut = int(timesteps*0.95) #time to consider for equilibration

In [11]:
# Data output 
SAVE_SIM = True
path_to_save_sim = 'placeholder/'

SAVE_STAT = True 
path_to_save_stat = 'placeholder/'

In [12]:
# Create folders to store data and parammeters list
fold0 = 'data_critical_line_N_sam{}_dt{}_timesteps{}_N_cut{}/'.format(N_sam, dt, timesteps, N_cut)

if SAVE_SIM:
    os.mkdir(path_to_save_sim+fold0)

if SAVE_STAT:
    os.mkdir(path_to_save_stat+fold0)


parameters = []

for S in S_list:
    fold1='S{}/'.format(S)

    if SAVE_SIM:
        os.mkdir(path_to_save_sim+fold0+fold1)

    if SAVE_STAT:
        os.mkdir(path_to_save_stat+fold0+fold1)

    for mu in mu_list:
        for sigma in sigma_list:
            for gamma in gamma_list:
                for K in K_list:
                    fold2 = 'mu{}_sigma{}_gamma{}_K{}/'.format(mu, sigma, gamma, K)
                    if SAVE_SIM:
                        os.mkdir(path_to_save_sim+fold0+fold1+fold2)

                    if SAVE_STAT:
                        os.mkdir(path_to_save_stat+fold0+fold1+fold2)

                    for i in range(N_sam):
                        parameters.append([S, mu, sigma, gamma, K, i])

In [13]:
#Phase diagram simulations
print('Number of tasks:', len(parameters))
data = Parallel(n_jobs = -1, verbose = 11)(delayed(simulation)(par) for par in parameters)

Number of tasks: 180


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  6.3min
[Paralle